In [ ]:
import sys
import numpy as np
import keras
from keras.utils import Sequence
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import os
import imgaug as ia
from imgaug import augmenters as iaa
import cv2

In [ ]:
BATCH_SIZE = 20
SEED = 777
SHAPE = (512, 512, 4)
DIR = '.'
ia.seed(SEED)

In [ ]:
def getTrainDataset():
    
    path_to_train = DIR + '/train/'
    data = pd.read_csv(DIR + '/train.csv')

    paths = []
    labels = []
    
    for name, lbl in zip(data['Id'], data['Target'].str.split(' ')):
        y = np.zeros(28)
        for key in lbl:
            y[int(key)] = 1
        paths.append(os.path.join(path_to_train, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

def getTestDataset():
    
    path_to_test = DIR + '/test/'
    data = pd.read_csv(DIR + '/sample_submission.csv')

    paths = []
    labels = []
    
    for name in data['Id']:
        y = np.ones(28)
        paths.append(os.path.join(path_to_test, name))
        labels.append(y)

    return np.array(paths), np.array(labels)

In [ ]:
# credits: https://github.com/keras-team/keras/blob/master/keras/utils/data_utils.py#L302
# credits: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

# Since our data set has samples which are really heavy, here we implement a custom data generator for fast 
# on-fly data loading. With cache = True, we let the Data Generator save all the data into RAM and then use 
# these in the following epochs.

class ProteinDataGenerator(keras.utils.Sequence):
            
    def __init__(self, paths, labels, batch_size, shape, shuffle = False, use_cache = False):
        self.paths, self.labels = paths, labels
        self.batch_size = batch_size
        self.shape = shape
        self.shuffle = shuffle
        self.use_cache = use_cache
        if use_cache == True:
            self.cache = np.zeros((paths.shape[0], shape[0], shape[1], shape[2]), dtype=np.float16)
            self.is_cached = np.zeros((paths.shape[0]))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size : (idx+1) * self.batch_size]

        paths = self.paths[indexes]
        X = np.zeros((paths.shape[0], self.shape[0], self.shape[1], self.shape[2]))
        
        # Generate data
        if self.use_cache == True:
            X = self.cache[indexes]
            for i, path in enumerate(paths[np.where(self.is_cached[indexes] == 0)]):
                image = self.__load_image(path)
                self.is_cached[indexes[i]] = 1
                self.cache[indexes[i]] = image
                X[i] = image
        else:
            for i, path in enumerate(paths):
                X[i] = self.__load_image(path)

        y = self.labels[indexes]
                
        return X, y
    
    def on_epoch_end(self):
        
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __iter__(self):
        """Create a generator that iterate over the Sequence."""
        for item in (self[i] for i in range(len(self))):
            yield item
            
    def __load_image(self, path):
        R = Image.open(path + '_red.png')
        G = Image.open(path + '_green.png')
        B = Image.open(path + '_blue.png')
        Y = Image.open(path + '_yellow.png')

        im = np.stack((
            np.array(R), 
            np.array(G), 
            np.array(B),
            np.array(Y)), -1)
        
        im = cv2.resize(im, (SHAPE[0], SHAPE[1]))
        im = np.divide(im, 255)
        return im

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf
from tensorflow import set_random_seed

set_random_seed(SEED)

In [ ]:
# Defining Macro F1 Score compatible with Keras

def f1(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# Defining Macro F1 Score loss function, accordingly to what is described on the report, compatible with Keras
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1-K.mean(f1)

In [ ]:
# Our model, based on GapNet

def create_model(input_shape):
    
    dropRate = 0.4
    
    init = Input(input_shape)
    x = BatchNormalization(axis=-1)(init)
    x = Conv2D(32, (3, 3))(x) #, strides=(2,2))(x)
    x = ReLU()(x)

    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp1 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp1)
    x = Conv2D(64, (3, 3), strides=(2,2))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(64, (3, 3))(x)
    x = ReLU()(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    ginp2 = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(ginp2)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    x = BatchNormalization(axis=-1)(x)
    x = Conv2D(128, (3, 3))(x)
    x = ReLU()(x)
    ginp3 = Dropout(dropRate)(x)
    
    gap1 = GlobalAveragePooling2D()(ginp1)
    gap2 = GlobalAveragePooling2D()(ginp2)
    gap3 = GlobalAveragePooling2D()(ginp3)
    
    x = Concatenate()([gap1, gap2, gap3])
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropRate)(x)
    
    x = BatchNormalization(axis=-1)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

In [ ]:
model = create_model(SHAPE)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-03),
    metrics=['acc',f1])


model.save('./BEST_kFold/0')
model.summary()

In [ ]:
# TRAINING + K-FOLD VALIDATION!

paths, labels = getTrainDataset()

np.random.seed(SEED)

epochs = 50
use_multiprocessing = False 
workers = 1                  

k = 3
l = int(paths.shape[0] / k)

for i in range(k):

    pathsTrain = np.concatenate(paths[:i*l], paths[(i+1)*l:])
    labelsTrain = np.concatenate([labels[:i*l], labels[(i+1)*l:]])
    pathsVal = paths[i*l:(i+1)*l]
    labelsVal = labels[i*l:(i+1)*l]
    
    print ("Generated Dimensions:")
    print(paths.shape, labels.shape)
    print(pathsTrain.shape, labelsTrain.shape, pathsVal.shape, labelsVal.shape)

    tg = ProteinDataGenerator(pathsTrain, labelsTrain, BATCH_SIZE, SHAPE, use_cache=False, shuffle = False)
    vg = ProteinDataGenerator(pathsVal, labelsVal, BATCH_SIZE, SHAPE, use_cache=False, shuffle = False)
    
    #Raw model from cell above
    model = load_model ('./BEST_kFold/0')
    
    directory = './BEST_kFold/{epoch:02d}-{val_loss:.6f}-{loss:.6f}-i:' + str(i)
    checkpoint = ModelCheckpoint(directory, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='min', period=1)
    reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')
    
    hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=1,
    callbacks=[checkpoint, reduceLROnPlato])
    

    # Plotting loss and accuracy over epochs
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('Loss Over Epochs')
    ax[0].plot(hist.epoch, hist.history["loss"], label="Train loss")
    ax[0].plot(hist.epoch, hist.history["val_loss"], label="Validation loss")
    ax[1].set_title('Accuracy Over Epochs')
    ax[1].plot(hist.epoch, hist.history["f1"], label="Train F1")
    ax[1].plot(hist.epoch, hist.history["val_f1"], label="Validation F1")
    ax[0].legend()
    ax[1].legend()
    path_fig = './BEST_kFold/plots' + ' - i:' + str(k) + '.png'
    fig.savefig(path_fig)  

In [ ]:
# FINE-TUNING!

# Choose which model to fine-tune:
filepath = ########
model = load_model(filepath, custom_objects={'f1': f1}) # in case you used the f1_loss, you should add
                                                        # , 'f1_loss': f1_loss}) there

savepath = #####
checkpoint2 = ModelCheckpoint(savepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='min', period=1)
reduceLROnPlato2 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='min')

epochs = 60

for layer in model.layers:
    layer.trainable = False
    
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True
model.layers[-7].trainable = True

model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=1e-4),
            metrics=['accuracy', f1])

#define tg and vg again, if necessary!

hist = model.fit_generator(
    tg,
    steps_per_epoch=len(tg),
    validation_data=vg,
    validation_steps=8,
    epochs=epochs,
    use_multiprocessing=False, 
    workers=1,
    verbose=1,
    max_queue_size=4,
    callbacks=[checkpoint2, reduceLROnPlato2]
)

fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].set_title('Loss Over Epochs')
ax[0].plot(hist.epoch, hist.history["loss"], label="Train loss")
ax[0].plot(hist.epoch, hist.history["val_loss"], label="Validation loss")
ax[1].set_title('Accuracy Over Epochs')
ax[1].plot(hist.epoch, hist.history["f1"], label="Train F1")
ax[1].plot(hist.epoch, hist.history["val_f1"], label="Validation F1")
ax[0].legend()
ax[1].legend()
fig.savefig(path_fig)

In [ ]:
# Preparing results 

# Load model after pure training (no finetuning)
bestModel = load_model(####, custom_objects={'f1': f1}) #, 'f1_loss': f1_loss})

# Load model after finetuning    
model = load_model(####, custom_objects={'f1': f1}) #, 'f1_loss': f1_loss})

# Use the whole validation set (or the set you want)
fullValGen = vg

from sklearn.metrics import f1_score as off1

# Obtain the threshold (binary classification problem for each class) for each class that maximizes its f1_score on 
# the validation set. T is the array where each entry corresponds to the best threshold for that class. It is obtained 
# by increasing by 0.001 from 0 to 1 its value and calculating the f1_score for each class, saving the best one. 
# Besides returning this array T, it still returns the macro f1_score for the validation set using those thresholds.
def getOptimalT(mdl, fullValGen):
    
    lastFullValPred = np.empty((0, 28))
    lastFullValLabels = np.empty((0, 28))
    for i in tqdm(range(len(fullValGen))): 
        im, lbl = fullValGen[i]
        scores = mdl.predict(im)
        lastFullValPred = np.append(lastFullValPred, scores, axis=0)
        lastFullValLabels = np.append(lastFullValLabels, lbl, axis=0)
    print(lastFullValPred.shape, lastFullValLabels.shape)
    
    rng = np.arange(0, 1, 0.001)
    f1s = np.zeros((rng.shape[0], 28))
    for j,t in enumerate(tqdm(rng)):
        for i in range(28):
            p = np.array(lastFullValPred[:,i]>t, dtype=np.int8)
            scoref1 = off1(lastFullValLabels[:,i], p, average='binary')
            f1s[j,i] = scoref1
    
    plt.plot(rng, f1s)
    T = np.empty(28)
    for i in range(28):
        T[i] = rng[np.where(f1s[:,i] == np.max(f1s[:,i]))[0][0]]
    
    #in order to know the distribution of f1_scores per class
    f1_score_p_class = np.max(f1s, axis=0)
    print('f1_score for each class is:')
    print(f1_score_p_class)
    
    return T, np.mean(f1_score_p_class)

# Calculate T and macro f1_score for each of the chosen models
print('Best pure training (no finetuning) model:')
T2, ff2 = getOptimalT(bestModel, fullValGen)

print('Last model after fine-tuning:')
T1, ff1 = getOptimalT(model, fullValGen)

print ('raw model - macrof1_score:')
print (ff2)
print ('model after fine-tuning - macrof1_score:')
print (ff1)

#We'll use the model with higher macro f1_score on the validation set
if ff1 > ff2:
    T = T1
    BEST = model
else:
    T = T2
    BEST = bestModel
    

In [ ]:
#Prediction on the test set
pathsTest, labelsTest = getTestDataset()

testg = ProteinDataGenerator(pathsTest, labelsTest, BATCH_SIZE, SHAPE)
submit = pd.read_csv(DIR + '/sample_submission.csv')
P = np.zeros((pathsTest.shape[0], 28))

for i in tqdm(range(len(testg))):
    images, labels = testg[i]
    score = BEST.predict(images)
    P[i*BATCH_SIZE:i*BATCH_SIZE+score.shape[0]] = score

PP = np.array(P)
prediction = []
for row in tqdm(range(submit.shape[0])):
    
    str_label = ''
    
    for col in range(PP.shape[1]):
        if(PP[row, col] < T[col]):
            str_label += ''
        else:
            str_label += str(col) + ' '
    prediction.append(str_label.strip())
    
submit['Predicted'] = np.array(prediction)
submit.to_csv('results_best_100epochs.csv', index=False)